In [22]:
import SimpleITK as sitk
import numpy as np
import os
import random
from glob import glob
import pandas as pd
import scipy.ndimage
from py import mhd_utils_3d
import matplotlib.pyplot as plt
import fnmatch

%matplotlib inline

In [10]:
luna_path = './'
luna_subset_path = luna_path + 'sample_patients/'
file_list = glob(luna_subset_path + "*.mhd")

annotations = pd.read_csv(luna_path + 'csv_files/' + 'annotations.csv')

if os.path.isdir('./npy'):
    pass
else:
    os.mkdir('./npy')

In [11]:
def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)

# map file full path to each record 
annotations['serisuidComplete'] = annotations['seriesuid'].map(lambda file_name: get_filename(file_list, file_name))
annotations = annotations.dropna()
annotations.head(5)

,seriesuid,coordX,coordY,coordZ,diameter_mm,serisuidComplete
1002,1.3.6.1.4.1.14519.5.2.1.6279.6001.487268565754...,119.208776,11.450374,-165.039862,26.837081,./sample_patients/1.3.6.1.4.1.14519.5.2.1.6279...
1003,1.3.6.1.4.1.14519.5.2.1.6279.6001.487268565754...,-37.901617,55.027912,-80.002605,17.718729,./sample_patients/1.3.6.1.4.1.14519.5.2.1.6279...
1145,1.3.6.1.4.1.14519.5.2.1.6279.6001.910435939545...,68.243850,83.940751,-296.892458,7.068564,./sample_patients/1.3.6.1.4.1.14519.5.2.1.6279...
1146,1.3.6.1.4.1.14519.5.2.1.6279.6001.910435939545...,53.469331,86.958564,-244.223244,8.704938,./sample_patients/1.3.6.1.4.1.14519.5.2.1.6279...
1147,1.3.6.1.4.1.14519.5.2.1.6279.6001.910435939545...,-91.369924,-6.167830,-218.833869,6.162973,./sample_patients/1.3.6.1.4.1.14519.5.2.1.6279...


In [12]:
print '数据条数：'
print df_node.shape[0]

数据条数：
9


In [18]:
# Some Functions
def getPathSeriesuid(seriesuid):
    pattern = '*'+seriesuid+'*'
    for roots, dirs, files in os.walk(luna_subset_path):
            for filename in fnmatch.filter(files,pattern):
                returnPath = os.path.join(roots,filename)
                #print(returnPath)
    return returnPath

def getAnnotationsFullPathCSV():
    # Includes full path of files in final column
    annotations["serisuidComplete"] = annotations["seriesuid"].apply(getPathSeriesuid) # Get complete paths of files (takes a while)
    annotations.to_csv("/CSVFILES/annotationsFullPaths.csv",index=0)

def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

def load_originSpacing(filename):
    itkimage = sitk.ReadImage(filename)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyOrigin, numpySpacing

'''
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(itkimage.GetOrigin()))
    numpySpacing = np.array(list(itkimage.GetSpacing()))
    return numpyImage, numpyOrigin, numpySpacing
'''

def worldToVoxelCoord(worldCoord, origin, spacing):
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord
#ofInterest = annotations[annotations["seriesuid"].str.contains("1.3.6.1.4.1.14519.5.2.1.6279.6001.105495028985881418176186711228")]

In [19]:
def getImageNVC(obs,data):
    data = data.iloc[obs]
    noduleCoords = data['coordX'],data['coordY'],data['coordZ']
    mhdPath = data.serisuidComplete.replace(".raw",".mhd")
    imgOrigin, imgSpacing = load_originSpacing(mhdPath)

    nvc = worldToVoxelCoord(noduleCoords[::-1],imgOrigin,imgSpacing)
    nvc = np.array(nvc.astype(int))
    return nvc

In [20]:
# Append Vox coords to dataframe
def makeAnnotationsCSVnodules(data,name):
    data['vxX'] = pd.Series(np.zeros(annotations.shape[0]), index=annotations.index)
    data['vxY'] = pd.Series(np.zeros(annotations.shape[0]), index=annotations.index)
    data['vxZ'] = pd.Series(np.zeros(annotations.shape[0]), index=annotations.index)
    for obs in xrange(data.shape[0]):
        #print(annotations.shape[0])
        if obs % 100 ==0:
            print(obs)
        nvc = getImageNVC(obs)

        data.set_value(obs, 'vxX', nvc[2])
        data.set_value(obs, 'vxY', nvc[1])
        data.set_value(obs, 'vxZ', nvc[0])
    data.to_csv("/home/msmith/luna16/CSVFILES/"+name+"NVC.csv",index=0) # just NVC

In [23]:
# Get random annotation from list
randInt = np.random.randint(annotations.shape[0])
#randInt = 1
annotationsOld = pd.read_csv(luna_path + 'csv_files/' + 'annotations.csv')
randAnnotation = annotationsOld.iloc[randInt]

noduleCoords = randAnnotation['coordX'],randAnnotation['coordY'],randAnnotation['coordZ']
pattern = '*'+randAnnotation['seriesuid']+'*'

imgPath = getPathSeriesuid(pattern)
rawPath = imgPath[:-3] + 'raw'
img = np.fromfile(rawPath,dtype='int16')
with open(imgPath[:-3] + 'mhd',"r") as f:
    imgData = f.read()
    mhdPath = rawPath[:-3] + 'mhd'
    
img, imgOrigin, imgSpacing = load_itk_image(mhdPath)

def printInfo():
    print("Using observation", randInt)
    print(randAnnotation)
    print("=="*50)
    print("=="*50)
    print(imgData)
    annotationsOld[randInt:randInt+1]
printInfo()

# Nodule voxel coordinates (nvc)
nvc = worldToVoxelCoord(noduleCoords[::-1],imgOrigin,imgSpacing)
nvc = np.array(nvc.astype(int))

#Get a range in which to subset the data
margin = 50
nvcMinus,nvcPlus = (nvc - margin).astype(int), (nvc + margin).astype(int)
def printNoduleInfo():
    print("Image shape ==>" + "\n" + str(img.shape))
    print("Nodule coordinates @"+"\n" + str(nvc))
    print("NVC margins ==>" + "\n" + str(nvcMinus) + "\n" + str(nvcPlus))
#printNoduleInfo()

from matplotlib.patches import Rectangle
rectangle = plt.Rectangle((nvcMinus[2], nvcMinus[1]),margin*2 ,margin*2, facecolor = 'none', ec='r')
circle = plt.Circle((nvc[2],nvc[1]),10, facecolor = 'none', ec='r')
plt.gca().add_patch(rectangle)
plt.gca().add_patch(circle)
plt.imshow(img[nvc[0]],cmap=cm.gray)
plt.show()

for i in xrange(nvc[0]-2,nvc[0]+2,1):
    if i == nvc[0]:
        print("Nodule located here")
    print(i)
    plt.imshow(img[i,nvcMinus[1]:nvcPlus[1],nvcMinus[2]:nvcPlus[2]],cmap=cm.gray)
    plt.show()

UnboundLocalError: local variable 'returnPath' referenced before assignment